In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from scipy.ndimage import gaussian_filter1d

from scipy import stats
import statsmodels.formula.api as smf

# import Python3_OpenOE_AC_map_functions_v1_08_30s as oem
import mz_LFP_functions as mz_LFP

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
sns.set_context('poster')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    # optional, makes markers bigger, too, axes.linewidth doesn't seem to work
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
    'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
    #'axes.color_cycle':pal, # image.cmap - rewritesd the default colormap
    'axes.linewidth':2.5, 'lines.linewidth': 2.5,
    'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black','axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

#set_pub_bargraphs()
set_pub_plots()

In [ ]:
pal=sns.blend_palette(['black','royalblue'],2)
sns.palplot(pal)
sns.set_palette(pal)

# Load some necessary variables

In [4]:
insert_depth = 1000  #change this as appropriate

sp_bw_ch = 20/2
samples_tr = 7350 #this is based on the shortest #samples in a trial
sr = 2500
n_chan = 384
rec_length = 3.0 #how long is the arduino triggered?

---

# First, load in the .npy arrays and CC_ls
These were creaded and saved using the "1_saving_LFP_arrays" jupyter notebook

In [5]:
all_pre_arr = np.load(r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\pre_all.npy")
all_post_arr = np.load(r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\post_all.npy")
all_novel_arr = np.load(r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\novel_all.npy")

In [ ]:
pkl_file = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\pre_et_ls"

open_file = open(pkl_file, "rb")
et_ls_pre = pickle.load(open_file)
open_file.close()

len(et_ls_pre)

In [ ]:
pkl_file = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\post_et_ls"

open_file = open(pkl_file, "rb")
et_ls_post = pickle.load(open_file)
open_file.close()

len(et_ls_post)

In [ ]:
pkl_file = r"D:\mz_Data\saved_dfs\HPC_nmda\lfp_npy\novel_et_ls"

open_file = open(pkl_file, "rb")
et_ls_novel = pickle.load(open_file)
open_file.close()

len(et_ls_novel)

---

# Plot the VEP ch trace

### First, we have to define a function

In [9]:
def VEP_lines(data_array, et_ls): #use this to separate into nmda & sham groups
    nmda = []
    sham = []
    for i in range(data_array.shape[0]):
        nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
        if et_ls[i] in nmda_ls:
            nmda.append(data_array[i])
        else:
            sham.append(data_array[i])

    nmda_arr = np.array(nmda)
    sham_arr = np.array(sham)

    mean_nmda = nmda_arr.mean(axis=0)
    mean_sham = sham_arr.mean(axis=0)

    print('Group nmda array: {0}'.format(nmda_arr.shape))
    print('Group nmda mean: {0}'.format(mean_nmda.shape))

    print('Group sham array: {0}'.format(sham_arr.shape))
    print('Group sham mean: {0}'.format(mean_sham.shape))

    V1_nmda = mean_nmda[0:100, :]
    min_nmda = np.where(V1_nmda == np.amin(V1_nmda))
    min_ch_nmda = min_nmda[0][0]
    print(min_ch_nmda)

    V1_sham = mean_sham[0:100, :]
    min_sham = np.where(V1_sham == np.amin(V1_sham))
    min_ch_sham = min_sham[0][0]
    print(min_ch_sham)
    
    return mean_nmda, mean_sham, min_ch_nmda, min_ch_sham

### Second, apply the function to the three situations

In [ ]:
# Pre
pre_mean_nmda, pre_mean_sham, pre_minch_nmda, pre_minch_sham = VEP_lines(all_pre_arr, et_ls_pre)

In [ ]:
# Post
post_mean_nmda, post_mean_sham, post_minch_nmda, post_minch_sham = VEP_lines(all_post_arr, et_ls_post)

In [ ]:
# Novel
novel_mean_nmda, novel_mean_sham, novel_minch_nmda, novel_minch_sham = VEP_lines(all_novel_arr, et_ls_novel)

## Aside, can check individual mice if you want!

In [13]:
def plot_mouse_Vep(array, channel, group, plt_title):
    plt.figure(figsize=(10, 5))
    if group == 'sham':
        plt_color = 'grey'
    elif group == 'nmda':
        plt_color = 'lightblue'
    mean_ch_traceA = array[channel,:]
    time_arr2_A = np.linspace(0, mean_ch_traceA.shape[0]/sr, mean_ch_traceA.shape[0])
    plt.plot(time_arr2_A, mean_ch_traceA, label=group, color=plt_color)
    plt.title(plt_title)
    plt.xlabel('Time (s)')
    plt.ylabel('uV')
    plt.ylim([-400,300])
    plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='b')
    plt.show()
    
def VEP_lines_all_mice(data_array, et_ls):
    nmda = []
    sham = []
    for i in range(data_array.shape[0]):
        nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
        if et_ls[i] in nmda_ls:
            nmda.append(data_array[i])
        else:
            sham.append(data_array[i])
    nmda_arr = np.array(nmda)
    sham_arr = np.array(sham)
    return nmda_arr, sham_arr

def single_mouse_Veps(big_array, avg_ch, my_group, plt_title):
    for mouse, m_arr in enumerate(big_array):
        print('Mouse index {}'.format(mouse))
        plot_mouse_Vep(m_arr, avg_ch, my_group, plt_title)

In [14]:
# Pre
pre_all_nmda, pre_all_sham = VEP_lines_all_mice(all_pre_arr, et_ls_pre)
# Post
post_all_nmda, post_all_sham = VEP_lines_all_mice(all_post_arr, et_ls_post)
# Novel
novel_all_nmda, novel_all_sham = VEP_lines_all_mice(all_novel_arr, et_ls_novel)

In [ ]:
# Pre Training individual mice
single_mouse_Veps(pre_all_nmda, pre_minch_nmda, my_group='nmda', plt_title='NMDA - Pre')
single_mouse_Veps(pre_all_sham, pre_minch_sham, my_group='sham', plt_title='Sham - Pre')

In [ ]:
# Post Training individual mice
single_mouse_Veps(post_all_nmda, post_minch_nmda, my_group='nmda', plt_title='NMDA - post')
single_mouse_Veps(post_all_sham, post_minch_sham, my_group='sham', plt_title='Sham - post')

In [ ]:
# Novel individual mice
single_mouse_Veps(novel_all_nmda, novel_minch_nmda, my_group='nmda', plt_title='NMDA - novel')
single_mouse_Veps(novel_all_sham, novel_minch_sham, my_group='sham', plt_title='Sham - novel')

### plot representative Veps for single mice

In [ ]:
plot_mouse_Vep(pre_all_nmda[3], pre_minch_nmda, group='nmda', plt_title='nmda - Pre')
plot_mouse_Vep(pre_all_sham[4], pre_minch_sham, group='sham', plt_title='sham - Pre')

In [ ]:
plot_mouse_Vep(post_all_nmda[3], post_minch_nmda, group='nmda', plt_title='nmda - post')
plot_mouse_Vep(post_all_sham[4], post_minch_sham, group='sham', plt_title='sham - post')

In [ ]:
plot_mouse_Vep(novel_all_nmda[3], novel_minch_nmda, group='nmda', plt_title='nmda - novel')
plot_mouse_Vep(novel_all_sham[4], novel_minch_sham, group='sham', plt_title='sham - novel')

# Otherwise, you can plot the group average graphs

In [ ]:
plt.figure(figsize=(10, 5))

pre_mean_ch_traceA = pre_mean_sham[pre_minch_sham,:]
pre_time_arr2_A = np.linspace(0, pre_mean_ch_traceA.shape[0]/sr, pre_mean_ch_traceA.shape[0])
pre_mean_ch_traceB = pre_mean_nmda[pre_minch_nmda,:]
pre_time_arr2_B = np.linspace(0, pre_mean_ch_traceB.shape[0]/sr, pre_mean_ch_traceB.shape[0])

plt.plot(pre_time_arr2_A, pre_mean_ch_traceA, label='sham')
plt.plot(pre_time_arr2_B, pre_mean_ch_traceB, label='nmda')
plt.legend(loc="lower right")

plt.title('Pre-training')
plt.xlabel('Time (s)')
plt.ylabel('uV')
plt.ylim([-350,250])
plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='grey')

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\lfp\pre_VEP_trace.pdf", transparent=True) # fix this before running

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

post_mean_ch_traceA = post_mean_sham[post_minch_sham,:]
post_time_arr2_A = np.linspace(0, post_mean_ch_traceA.shape[0]/sr, post_mean_ch_traceA.shape[0])
post_mean_ch_traceB = post_mean_nmda[post_minch_nmda,:]
post_time_arr2_B = np.linspace(0, post_mean_ch_traceB.shape[0]/sr, post_mean_ch_traceB.shape[0])

plt.plot(post_time_arr2_A, post_mean_ch_traceA, label='sham')
plt.plot(post_time_arr2_B, post_mean_ch_traceB, label='nmda')
plt.legend(loc="lower right")

plt.title('Post-training')
plt.xlabel('Time (s)')
plt.ylabel('uV')
plt.ylim([-350,250])
plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='grey')

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\lfp\post_VEP_trace.pdf", transparent=True) # fix this before running

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

novel_mean_ch_traceA = novel_mean_sham[novel_minch_sham,:]
novel_time_arr2_A = np.linspace(0, novel_mean_ch_traceA.shape[0]/sr, novel_mean_ch_traceA.shape[0])
novel_mean_ch_traceB = novel_mean_nmda[novel_minch_nmda,:]
novel_time_arr2_B = np.linspace(0, novel_mean_ch_traceB.shape[0]/sr, novel_mean_ch_traceB.shape[0])

plt.plot(novel_time_arr2_A, novel_mean_ch_traceA, label='sham')
plt.plot(novel_time_arr2_B, novel_mean_ch_traceB, label='nmda')
plt.legend(loc="lower right")

plt.title('Novel')
plt.xlabel('Time (s)')
plt.ylabel('uV')
plt.ylim([-350,250])
plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='grey')

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\lfp\novel_VEP_trace.pdf", transparent=True) # fix this before running

plt.show()

---

In [ ]:
plt.plot(post_time_arr2_A, post_mean_ch_traceA, label='sham')
plt.plot(post_time_arr2_B, post_mean_ch_traceB, label='nmda')

plt.axvspan(0.55, 0.77, alpha=0.2)
plt.axvspan(0.78, 0.95, alpha=0.2)
plt.axvspan(0.96, 1.2, alpha=0.2)
plt.axvspan(1.21, 1.45, alpha=0.2)
plt.axvspan(1.46, 1.8, alpha=0.2)

plt.xlim([0.5,2])

# Finding the VEP peaks in each situation

In [23]:
time_win=[[0.55, 0.77],   # VEP 1 time frame
          [0.78, 0.95],  # VEP 2 time frame
          [0.96, 1.2],     # VEP 3 time frame
          [1.21, 1.45],  # VEP 4 time frame
          [1.46, 1.8]]    # VEP 5 time frame


time_win=[[i[0]*sr,i[1]*sr] for i in time_win]
depth_b=[[0,100],[101,102]] #V1 bounds and hippocampus/Thalamus bounds


In [ ]:
pre_V1vepdf = mz_LFP.create_vepamp_df(all_pre_arr, time_windows=time_win, depth_bounds=depth_b)
post_V1vepdf = mz_LFP.create_vepamp_df(all_post_arr, time_windows=time_win, depth_bounds=depth_b)
novel_V1vepdf = mz_LFP.create_vepamp_df(all_novel_arr, time_windows=time_win, depth_bounds=depth_b)

# pre_V1vepdf = mz_LFP.create_vep_df(all_pre_arr, time_windows=time_win, depth_bounds=depth_b)
# post_V1vepdf = mz_LFP.create_vep_df(all_post_arr, time_windows=time_win, depth_bounds=depth_b)
# novel_V1vepdf = mz_LFP.create_vep_df(all_novel_arr, time_windows=time_win, depth_bounds=depth_b)

# r1_V1vepdf.head()

In [ ]:
novel_V1vepdf.head()

### Second, we add in some additonal info to the dfs

In [64]:
et_ls = et_ls_pre

et_dfls = []
groupls = []
for i in pre_V1vepdf.rec_ind:
    et_dfls.append(et_ls[i])
    nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
    if et_ls[i] in nmda_ls:
        groupls.append("nmda")
    else:
        groupls.append("sham")

pre_V1vepdf["et"] = et_dfls
pre_V1vepdf["group"] = groupls
pre_V1vepdf["stage"] = 'pre'

In [65]:
et_ls = et_ls_post

et_dfls = []
groupls = []
for i in post_V1vepdf.rec_ind:
    et_dfls.append(et_ls[i])
    nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
    if et_ls[i] in nmda_ls:
        groupls.append("nmda")
    else:
        groupls.append("sham")

post_V1vepdf["et"] = et_dfls
post_V1vepdf["group"] = groupls
post_V1vepdf["stage"] = 'post'

In [66]:
et_ls = et_ls_novel

et_dfls = []
groupls = []
for i in novel_V1vepdf.rec_ind:
    et_dfls.append(et_ls[i])
    nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
    if et_ls[i] in nmda_ls:
        groupls.append("nmda")
    else:
        groupls.append("sham")

novel_V1vepdf["et"] = et_dfls
novel_V1vepdf["group"] = groupls
novel_V1vepdf["stage"] = 'novel'

In [67]:
all_training = pd.concat([pre_V1vepdf, post_V1vepdf, novel_V1vepdf])

### Finally, we can plot the quantification results

In [ ]:
sns.catplot(data=all_training,
            x='vep',
            y='vepamp',
            kind='point',
            hue='group',
            col='stage',
            ci=68,
            legend=False,
            hue_order=['sham','nmda'],
            height = 5,
            aspect=1.2
           )

plt.legend(loc="lower right")
plt.ylabel('uV')
plt.xlabel('Oscillation Cycle')
# plt.ylim([0,500])
# plt.yticks([0,250,500])

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\lfp\VEPamp_quant.pdf", transparent=True) # fix this before running

plt.show()

---

# Testing the stats about the above plots
### Overall test for the genotypes (wt vs dko)

In [ ]:
pre_stats, pre_fstats = mz_LFP.VEP_overall_stats(pre_V1vepdf, group_ls=['sham','nmda'])
post_stats, post_fstats = mz_LFP.VEP_overall_stats(post_V1vepdf, group_ls=['sham','nmda'])
novel_stats, novel_fstats = mz_LFP.VEP_overall_stats(novel_V1vepdf, group_ls=['sham','nmda'])

In [ ]:
print('~~~~~ Pre training ~~~~~')
print(pre_stats)
print(pre_fstats)
print('~~~~~ Post training ~~~~~')
print(post_stats)
print(post_fstats)
print('~~~~~ Novel ~~~~~')
print(novel_stats)
print(novel_fstats)

### Individual tests for each vep peak

In [55]:
prepeak_stats, prepeak_fstats = mz_LFP.VEP_peaks_stats(pre_V1vepdf, group_ls=['sham','nmda'])
postpeak_stats, postpeak_fstats = mz_LFP.VEP_peaks_stats(post_V1vepdf, group_ls=['sham','nmda'])
novelpeak_stats, novelpeak_fstats = mz_LFP.VEP_peaks_stats(novel_V1vepdf, group_ls=['sham','nmda'])

In [ ]:
print('~~~~~ Pre training ~~~~~')
print(prepeak_stats)
print(prepeak_fstats)
print('~~~~~ Post training ~~~~~')
print(postpeak_stats)
print(postpeak_fstats)
print('~~~~~ Novel ~~~~~')
print(novelpeak_stats)
print(novelpeak_fstats)

### Linear Mixed Model analysis
First, for the Amp values x Vep # with a comparison between groups

Second, for the Amp values x group with a comparison between Vep #

In [ ]:
pre_mdf1, pre_mdf2 = mz_LFP.LMM_stats(pre_V1vepdf)
print(pre_mdf1.pvalues)
print(pre_mdf2.pvalues)

In [ ]:
post_mdf1, post_mdf2 = mz_LFP.LMM_stats(post_V1vepdf)
print(post_mdf1.pvalues)
print(post_mdf2.pvalues)

In [ ]:
novel_mdf1, novel_mdf2 = mz_LFP.LMM_stats(novel_V1vepdf)
print(novel_mdf1.pvalues)
print(novel_mdf2.pvalues)

---

---

---

---

---

In [60]:
#new method for determining the peaks, just get the troughs and values from that, don't use time windows
import scipy.signal as ss

In [200]:
def new_vep_amp(all_array, et_ls, training_stage):
    big_df_ls = []
    for mi,mouse in enumerate(all_array):
        min_arr = np.where(mouse[50:80, 1250:5000] == np.amin(mouse[50:80, 1250:5000]))
        min_ch = min_arr[0][0]+50

        mouse_vep = mouse[min_ch, 1250:5000] #find the VEP channel

        #find the idx of the peaks and troughs & find the associated value for each
        inv_vep = -mouse_vep
        troughs, _ = ss.find_peaks(inv_vep, distance=100, width=20, prominence=40)
        peaks, _ = ss.find_peaks(mouse_vep, distance=100, width=20, prominence=40)
        
        plt.plot(mouse_vep)
        plt.plot(peaks, mouse_vep[peaks], "x")
        plt.plot(troughs, mouse_vep[peaks], "x")
        
#         tr_values = []
#         for idx in troughs:
#             val = mouse_vep[idx]
#             tr_values.append(val)
#         pk_values = []
#         for idx in peaks:
#             val = mouse_vep[idx]
#             pk_values.append(val)
#         #calculate the amplitude of each trough-peak change
#         tr_pk = []
#         pk_idx = []
#         for ii, t_val in enumerate(tr_values):
#             try:
#                 p_val = pk_values[ii]
#             except:
#                 p_val = 0
#             tr_pk.append(abs(val - p_val))
#             pk_idx.append(ii+1)

#         #get metadata for the df below
#         nmda_ls = ['et1710', 'et1700', 'et1570', 'et1520', 'et171', 'et170', 'et157', 'et152']
#         if et_ls[mi] in nmda_ls:
#             group = "nmda"
#         else:
#             group = "sham"

#         #make dataframe of vep quantification
#         mouse_df = pd.DataFrame({'vep':pk_idx,'vepamp':tr_pk,'et':et_ls[mi], 'group':group, 'stage':training_stage})    
#         big_df_ls.append(mouse_df)

#     post_veps = pd.concat(big_df_ls)
    
#     return post_veps

In [ ]:
pre_veps_df = new_vep_amp(all_pre_arr, et_ls_pre, training_stage='pre')
post_veps_df = new_vep_amp(all_post_arr, et_ls_post, training_stage='post')
novel_veps_df = new_vep_amp(all_novel_arr, et_ls_novel, training_stage='novel')

all_training_veps = pd.concat([pre_veps_df, post_veps_df, novel_veps_df])

In [ ]:
sns.catplot(data=all_training_veps,
            x='vep',
            y='vepamp',
            kind='point',
            hue='group',
            col='stage',
            ci=68,
            legend=False,
            hue_order=['sham','nmda'],
            height = 4.5,
            aspect=1.5
           )

# plt.legend(loc="lower right")

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\LFP\pre_VEP_quant.pdf", transparent=True) # fix this before running

plt.show()